In [1]:
from zipfile import ZipFile
import numpy as np

In [3]:
import pandas as pd
import zipfile
import re

zf = zipfile.ZipFile('Data_incidents.csv.zip') 
df = pd.read_csv(zf.open('Data_incidents.csv.zip'))

BadZipFile: File is not a zip file